# Convert between 3D tiff and nifti images

In [1]:
# Automatically reload module code when it is changed
%load_ext autoreload
%autoreload 2

### Import modules

In [2]:
import pathlib as pl
from build_bird_template import tiff_to_nifti, nifti_to_tiff

### Configure paths

In [8]:
# Path to data for specific bird species (assert that folders exist)
species = 'BlackCap'
data_folder = pl.Path(f'/home/nsirmpilatze/Data/bird-templates/{species}')
assert data_folder.is_dir(), f'{data_folder} is not a valid directory'

# Where to store brain tiff files
data_tiff = data_folder / 'brains_tiff' 
data_tiff.mkdir(parents=True, exist_ok=True)
# Where to store brain tiff files
data_nifti = data_folder / 'brains_nifti' 
data_nifti.mkdir(parents=True, exist_ok=True)

## A. Tiff to Nifti

### Select animals, channels, and resolution 

In [6]:
# Animal IDs
# Initially we will only use the brains without any viral injections
animal_ids = ['SW_BC38', 'SW_BC58', 'SW_BC73', 'SW_BC74']

# Select fluorescent channel
channel_colors = {'ch02': 'red', 'ch03': 'green', 'ch04': 'blue'} 
# For non-injected animals, the red or green channel can be used
ch = 'ch03'
color = channel_colors[ch]

# Downsampled data are available at 3 resolutions at most
resolutions = {'010_micron': 10, '025_micron': 25 , '050_micron': 50}
# Let's start with the lowest resolution first
res_name = '050_micron'
pix_sizes = [resolutions[res_name]] * 3

### Convert tiff images to nifti

In [7]:
for animal_id in animal_ids:
    # tiff folder
    tiff_folder = data_tiff / animal_id / 'downsampled_stacks' / res_name
    assert tiff_folder.is_dir(), f'{tiff_folder} is not a valid directory'
    # Find valid tiff file in folder
    tiff_file = [f for f in tiff_folder.iterdir() if (f.suffix == '.tif') & (color in f.name)][0]
    # Define nifti file path
    nifti_file = data_nifti / f'{animal_id}_{ch}_{color}_{res_name}.nii'
    # Convert tiff to nifti
    tiff_to_nifti(tiff_file, nifti_file, pix_sizes)
    print(f'{tiff_file.name} converted to {nifti_file.name}')

ds_SW_BC38_210317_204349_50_50_ch03_green.tif converted to SW_BC38_ch03_green_050_micron.nii
ds_SW_BC58_220614_120516_50_50_ch03_chan_3_green.tif converted to SW_BC58_ch03_green_050_micron.nii
ds_SW_BC73_220614_120516_50_50_ch03_chan_3_green.tif converted to SW_BC73_ch03_green_050_micron.nii
ds_SW_BC74white_220217_120749_50_50_ch03_chan_3_green.tif converted to SW_BC74_ch03_green_050_micron.nii


## Nifti to Tiff
(After template is created by `Build Template.ipynb`)

### Add template path

In [10]:
template_version = 'v1'
template_folder = data_folder / f'template_{template_version}'
assert template_folder.is_dir(), f'{template_folder} is not a valid directory'

### Convert useful template outputs to tiff
This includes:
1. The final template image (a sharpened 'average' of all input brains)
2. The resample brain images into the template space (for asessing registration success)

In [46]:
final_template = template_folder / 'final' / 'average' / 'template_sharpen_shapeupdate.nii.gz'
resampled_brains_dir = template_folder / 'final' / 'resample' 
resampled_brains = [f for f in resampled_brains_dir.iterdir() if f.suffix == '.gz']

# Create new template subfolder for storing tiffs
template_tiff_dir = template_folder / 'results_tiff'
template_tiff_dir.mkdir(parents=True, exist_ok=True)

print('Converting final template image to tiff...')
template_tiff = template_tiff_dir / 'template.tif'
nifti_to_tiff(final_template, template_tiff)
print(f'\t{final_template.name} converted to {template_tiff.name}')

print('Converting resampled brains to tiff...')
for resampled_brain in resampled_brains:
    tiff_file = template_tiff_dir / f'{resampled_brain.stem.split(".")[0]}_resampled.tif'
    nifti_to_tiff(resampled_brain, tiff_file)
    print(f'\t{resampled_brain.name} converted to {tiff_file.name}')

Converting final template image to tiff...
	template_sharpen_shapeupdate.nii.gz converted to template.tif
Converting resampled brains to tiff...
	SW_BC38_ch03_green_050_micron.nii.gz converted to SW_BC38_ch03_green_050_micron_resampled.tif
	SW_BC58_ch03_green_050_micron.nii.gz converted to SW_BC58_ch03_green_050_micron_resampled.tif
	SW_BC73_ch03_green_050_micron.nii.gz converted to SW_BC73_ch03_green_050_micron_resampled.tif
	SW_BC74_ch03_green_050_micron.nii.gz converted to SW_BC74_ch03_green_050_micron_resampled.tif
